## Logistic Regression

In this section we will implement a logistic regression model trainable with SGD using numpy. Here are the objectives:

1. Implement a simple forward model with no hidden layer (equivalent to logistic regression):
$y = softmax(\mathbf{W} x + b)$

1. build a `predict` function which returns the most probable class given an input $x$

1. build an `accuracy` function for a batch of inputs $X$ and the corresponding expected outputs $y_{true}$

1. build a `grad` function which computes $\frac{d}{dW} -\log(softmax(Wx + b))$ for an $x$ and its corresponding expected output $y_{true}$ ; check that the gradients are well defined

1. build a `train` function which uses the `grad` function output to update $\mathbf{W}$ and $b$


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14

from sklearn.datasets import load_digits
import numpy as np

digits = load_digits()

First let's define a helper function to compute the one hot encoding of an integer array for a fixed number of classes:

In [2]:
def one_hot(n_classes, y):
    return np.eye(n_classes)[y]

In [3]:
one_hot(10, 3)

In [4]:
one_hot(10, [3,2,1,0])

In [5]:
one_hot(10, [0, 4, 9, 1])

Let's take a moment to take a look at the dataset before we start using it.

In [6]:
sample_index = 42 + 31 + 3 # change this to see different examples
plt.figure(figsize=(3, 3))
plt.imshow(digits.images[sample_index], cmap=plt.cm.gray_r,
           interpolation='nearest')
plt.title("image label: %d" % digits.target[sample_index]);

### Preprocessing

- normalization (for more take a look at http://scikit-learn.org/stable/modules/preprocessing.html)
- train/test split

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


data = np.asarray(digits.data, dtype='float32')
target = np.asarray(digits.target, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.15, random_state=37)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Y_train = one_hot(10, y_train)
Y_test = one_hot(10, y_test)

In [8]:
Y_train = one_hot(10, y_train)
Y_train[:3]

Let's display the one of the transformed sample (after feature standardization):

In [9]:
sample_index = 45
plt.figure(figsize=(3, 3))
plt.imshow(X_train[sample_index].reshape(8, 8),
           cmap=plt.cm.gray_r, interpolation='nearest')
plt.title("transformed sample\n(standardised)");

The scaler objects makes it possible to recover the original sample:

In [10]:
plt.figure(figsize=(3, 3))
plt.imshow(scaler.inverse_transform(X_train[sample_index]).reshape(8, 8),
           cmap=plt.cm.gray_r, interpolation='nearest')
plt.title("original sample");

Now let's implement the softmax vector function:

$$
softmax(\mathbf{x}) = \frac{1}{\sum_{i=1}^{n}{e^{x_i}}}
\cdot
\begin{bmatrix}
  e^{x_1}\\\\
  e^{x_2}\\\\
  \vdots\\\\
  e^{x_n}
\end{bmatrix}
$$

In [11]:
def softmax(X):
    # TODO:
    exp = np.exp(X)
    return exp / np.sum(exp, axis=-1, keepdims=True)

Make sure that this works one vector at a time (and check that the components sum to one):

In [12]:
print(softmax([10, 2, -3]))

Note that a naive implementation of softmax might not be able process a batch of activations in a single call (but we need that):

In [13]:
X = np.array([[10, 2, -3],
              [-1, 5, -20]])
print(softmax(X))

Implement a function that given the true one-hot encoded class `Y_true` and and some predicted probabilities `Y_pred` returns the negative log likelihood. The negative log likelihood also appears under the name "log loss" or "cross entropy".

$$
L = \sum_N \ell(y^{\mathsf{true}}, y^{\mathsf{pred}}) = \sum_N \sum_k \mathbf{1}_{y^{\mathsf{true}}_i = y^{\mathsf{pred}}_i} \log(p_i)
$$

This is a bit of a notational nightmare. Lots of articles only give the expression for two class problems but we need the ten class version. Take a look at the [wikipedia article](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression).

In [14]:
EPSILON = 1e-8

def nll(Y_true, Y_pred):
    # TODO
    # Fill in this function. You need two loops here
    # one over the possible classes per sample
    # and then one over all samples.
    # For the loop over classes you want to select
    # only that log(p) term that corresponds to the
    # correct class.
    return None

# Make sure that it works for a simple sample at a time
print(nll([1, 0, 0], [.99, 0.01, 0]))

Check that the `nll` of a very confident yet incorrect prediction is a much higher positive number:

In [15]:
print(nll([1, 0, 0], [0.01, 0.01, .98]))

Make sure that your implementation can compute the average negative log likelihood of a group of predictions: `Y_pred` and `Y_true` can therefore be passed in as 2D arrays:

In [16]:
def nll(Y_true, Y_pred):
    Y_true, Y_pred = np.atleast_2d(Y_true), np.atleast_2d(Y_pred)
    loglikelihoods = np.sum(np.log(EPSILON + Y_pred) * Y_true, axis=1)
    return -np.mean(loglikelihoods)

In [17]:
# Check that the average NLL of the following 3 almost perfect
# predictions is close to 0
Y_true = np.array([[0, 1, 0],
                   [1, 0, 0],
                   [0, 0, 1]])

Y_pred = np.array([[0,   0.99,    0],
                   [.99, 0.01, 0],
                   [0,   0,    1]])

print(nll(Y_true, Y_pred))

Finally we have all the ingredients for training a logistic regression model using gradient descent.

Let's study it one sample at a time.

In [18]:
class LogisticRegression():
    def __init__(self, input_size, output_size):
        self.W = np.random.uniform(size=(input_size, output_size),
                                   high=0.1, low=-0.1)
        self.b = np.random.uniform(size=output_size,
                                   high=0.1, low=-0.1)
        self.output_size = output_size
        
    def forward(self, X):
        # TODO: compute normalised scores
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict(self, X):
        # TODO: for each sample return the predicted class
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def grad_loss(self, x, y_true):
        # TODO?: compute gradient with respect to W and b for a sample x
        # and the true labels y_true
        # If your linear algebra is good try and derive the
        # expressions for the gradients from the loss function
        # (Your answer does not have to look this compact. It
        # has taken a while to optimise it down to such a
        # compact piece of code.)
        y_pred = self.forward(x)
        dnll_output =  y_pred - one_hot(self.output_size, y_true)
        grad_W = np.outer(x, dnll_output)
        grad_b = dnll_output

        grads = {"W": grad_W, "b": grad_b}
        return grads
    
    def train(self, x, y, learning_rate):
        # TODO:
        # Traditional gradient descent update without momentum
        grads = self.grad_loss(x, y)
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def loss(self, x, y):
        # TODO: use `nll` to compute the loss for the sample x with true label y
        # YOUR CODE HERE
        raise NotImplementedError()

    def accuracy(self, X, y):
        # TODO: compute accuracy for samples X with true labels y
        # YOUR CODE HERE
        raise NotImplementedError()

In [19]:
one_hot(10, y_train)

In [20]:
y_train

In [21]:
# Build a model and test its forward inference
# you can do this before you have implemented
# the gradient descent part. Check that the
# predictions work.
n_features = X_train.shape[1]
n_classes = Y_train.shape[1]
lr = LogisticRegression(n_features, n_classes)

print("Evaluation of the untrained model:")
train_loss = lr.loss(X_train, y_train)
train_acc = lr.accuracy(X_train, y_train)
test_acc = lr.accuracy(X_test, y_test)

print("train loss: %0.4f, train acc: %0.3f, test acc: %0.3f"
      % (train_loss, train_acc, test_acc))
# Question: what should the accuracy be for the untrained
# model?

In [22]:
# Test the untrained model on the first example
sample_idx = 3 + 45
plt.plot(lr.forward(X_train[sample_idx]), linestyle='-', label='prediction')
plt.plot(one_hot(10, y_train[sample_idx]), linestyle='--', label='true')
plt.title('output probabilities')
plt.legend()
print(lr.predict(X_train[sample_idx]))

In [23]:
# Implement training for one epoch using
# stochastic gradient descent. Print out
# the accuracy on training and testing set.
learning_rate = 0.01

# YOUR CODE HERE
raise NotImplementedError()

In [24]:
# Evaluate the trained model on an example
# What should you see if you compare the true
# and predicted y vectors for one sample?
# YOUR CODE HERE
raise NotImplementedError()

Questions:

* can you find examples that are mispredicted, is there a pattern to the wrong predictions?
* visualise the samples and predicted classes
* plot the [confusion matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) to classes that are hard to separate (maybe eight vs nine?)

In [25]:
# Evaluate the trained model on an example
# YOUR CODE HERE
raise NotImplementedError()